In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, util
import torch
from tqdm import tqdm
import math

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        texts = (row['question'], row['context'])
        return texts

# Load the dataset
csv_path = 'fine_tuning_dataset/all_merged_dataset.csv'
df = pd.read_csv(csv_path)

# Create a dataset and a DataLoader
train_dataset = CustomDataset(df)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Use MultipleNegativesRankingLoss for training
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune the model
num_epochs = 5
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# To store train losses
train_losses = []

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
        optimizer.zero_grad()
        
        # Tokenize the batch of text pairs
        texts = [example for example in batch]
        batch_features = model.tokenize(texts)
        
        # Move input tensors to the GPU
        batch_features = {key: val.to(device) for key, val in batch_features.items()}
        
        # Forward pass
        model_output = model(batch_features)
        
        # Compute loss
        loss_value = train_loss(model_output['sentence_embedding'])
        
        # Backward pass and optimization
        loss_value.backward()
        optimizer.step()
        
        # Accumulate loss value
        epoch_loss += loss_value.item()
    
    # Compute average loss for the epoch
    avg_loss = epoch_loss / len(train_dataloader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")

print("Model fine-tuning complete. Model saved to `output_path_to_save_model`.")
print("Training losses over epochs: ", train_losses)
model.save('output_path_to_save_model')

In [ ]:
# learning rate finder
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, util
import torch
import torch.optim as optim
import math
from sklearn.model_selection import train_test_split

# Load the dataset
csv_path = 'fine_tuning_dataset/all_merged_dataset.csv'
df = pd.read_csv(csv_path)

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Create InputExamples for training
train_examples = [InputExample(texts=[row['question'], row['context']]) for _, row in train_df.iterrows()]

# Create a DataLoader
batch_size = 16
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Use MultipleNegativesRankingLoss for training
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune the model
num_epochs = 10  # Example: Start with fewer epochs
initial_lr = 0.00002
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
questions = test_df['question'].values
contexts = test_df['context'].values

optimizer = optim.AdamW(model.parameters(), lr=initial_lr)

# Assuming you want to use a custom training loop for better control
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    
    model.zero_grad()
    for step, batch in enumerate(train_dataloader):
        inputs = batch
        loss = train_loss(inputs)
        loss.backward()
        optimizer.step()

    # Here you would typically evaluate the performance on a validation set
    # model_test = SentenceTransformer(output_path, device='cpu')

    # with torch.no_grad():
    #     question_embeddings = model.encode(questions, convert_to_tensor=True)
    #     context_embeddings = model.encode(contexts, convert_to_tensor=True)

    #     # Perform element-wise multiplication for each question-context pair
    #     combined_embeddings = torch.mul(question_embeddings, context_embeddings)

    #     # Calculate cosine similarity matrix for the combined embeddings
    #     similarity_matrix = util.cos_sim(combined_embeddings, combined_embeddings).cpu().numpy()

    # # Extract the upper triangular part of the similarity matrix, excluding the diagonal
    # triu_indices = np.triu_indices_from(similarity_matrix, k=1)
    # triu_values = similarity_matrix[triu_indices]
    # average_similarity_score = np.mean(triu_values)

    # print("\nAverage Similarity Score (excluding diagonal):")
    # print(average_similarity_score)
    # print("\n" + "="*50 + "\n")

    # torch.cuda.empty_cache()
    # torch.cuda.ipc_collect()
    # del model_test
    torch.cuda.empty_cache()

print("Training Complete.")

In [1]:
# learning rate
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, util
import math
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim

# Load the dataset
csv_path = 'fine_tuning_dataset/all_merged_dataset.csv'
df = pd.read_csv(csv_path)

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Create InputExamples for training
train_examples = [InputExample(texts=[row['question'], row['context']]) for _, row in train_df.iterrows()]

# Create a DataLoader
# default batch_size=16
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Use MultipleNegativesRankingLoss for training
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune the model
num_epochs = 300
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
questions = test_df['question'].values
contexts = test_df['context'].values

assert len(questions) == len(contexts), "The number of questions must match the number of contexts"

# Define the optimizer
# optimizer = optim.AdamW(model.parameters(), lr=0.00002)  # Initial learning rate
# Scheduler parameters can be defined in the fit method

# normal also as default: 2e-5
learning_rates = [2e-5, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]

lr_picked = learning_rates[0]
output_path = f'output_path_to_save_model-{lr_picked}'
for i in range(num_epochs):
    print(f"fine tune epoch {i+1}")
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=warmup_steps,
        output_path=output_path,
        # optimizer_class=optim.AdamW,
        # optimizer_params={'lr': lr_picked},
        scheduler='WarmupLinear'  # Use a supported scheduler
    )
    model_test = SentenceTransformer(output_path, device='cpu')

    with torch.no_grad():
        question_embeddings = model.encode(questions, convert_to_tensor=True)
        context_embeddings = model.encode(contexts, convert_to_tensor=True)

        # Perform element-wise multiplication for each question-context pair
        combined_embeddings = torch.mul(question_embeddings, context_embeddings)

        # Calculate cosine similarity matrix for the combined embeddings
        similarity_matrix = util.cos_sim(combined_embeddings, combined_embeddings).cpu().numpy()

    # Extract the upper triangular part of the similarity matrix, excluding the diagonal
    triu_indices = np.triu_indices_from(similarity_matrix, k=1)
    triu_values = similarity_matrix[triu_indices]
    average_similarity_score = np.mean(triu_values)

    print("\nAverage Similarity Score (excluding diagonal):")
    print(average_similarity_score)
    print("\n" + "="*50 + "\n")

    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    del model_test


print("Model fine-tuning complete. Model saved to `output_path_to_save_model`.")

/home/ram/Codes/college/chatbot/local-langchain/venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


fine tune epoch 1


/home/ram/Codes/college/chatbot/local-langchain/venv/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%|██████████| 80/80 [00:18<00:00,  4.31it/s]


{'train_runtime': 18.5636, 'train_samples_per_second': 34.099, 'train_steps_per_second': 4.31, 'train_loss': 0.580385971069336, 'epoch': 1.0}



Average Similarity Score (excluding diagonal):
0.29116905


fine tune epoch 2


...

{'train_runtime': 88.9796, 'train_samples_per_second': 7.114, 'train_steps_per_second': 0.899, 'train_loss': 4.6439261495834215e-06, 'epoch': 1.0}

Average Similarity Score (excluding diagonal):
0.24312511


fine tune epoch 192


/home/ram/Codes/college/chatbot/local-langchain/venv/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%|██████████| 80/80 [01:28<00:00,  1.11s/it]


{'train_runtime': 88.9849, 'train_samples_per_second': 7.114, 'train_steps_per_second': 0.899, 'train_loss': 4.483555312617682e-06, 'epoch': 1.0}

Average Similarity Score (excluding diagonal):
0.243313


fine tune epoch 193


/home/ram/Codes/college/chatbot/local-langchain/venv/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/80 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, util
import math
from sklearn.model_selection import train_test_split
import torch

# Load the dataset
# Modify the path to point to your csv file
csv_path = 'fine_tuning_dataset/all_merged_dataset.csv'
df = pd.read_csv(csv_path)

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Create InputExamples for training
train_examples = [InputExample(texts=[row['question'], row['context']]) for _, row in train_df.iterrows()]

# Create InputExamples for testing

# Create a DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Use MultipleNegativesRankingLoss for training
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune the model
num_epochs = 300
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
questions = test_df['question'].values
contexts = test_df['context'].values

assert len(questions) == len(contexts), "The number of questions must match the number of contexts"

output_path = 'output_path_to_save_model'
for i in range(num_epochs):
    print(f"fine tune epoch {i+1}")
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=warmup_steps,
        output_path=output_path,
        scheduler=scheduler
    )
    model_test = SentenceTransformer(output_path, device='cpu')

    with torch.no_grad():
        question_embeddings = model.encode(questions, convert_to_tensor=True)
        context_embeddings = model.encode(contexts, convert_to_tensor=True)

        # Perform element-wise multiplication for each question-context pair
        combined_embeddings = torch.mul(question_embeddings, context_embeddings)

        # Calculate cosine similarity matrix for the combined embeddings
        similarity_matrix = util.cos_sim(combined_embeddings, combined_embeddings).cpu().numpy()

    # Extract the upper triangular part of the similarity matrix, excluding the diagonal
    triu_indices = np.triu_indices_from(similarity_matrix, k=1)
    triu_values = similarity_matrix[triu_indices]
    average_similarity_score = np.mean(triu_values)

    print("\nAverage Similarity Score (excluding diagonal):")
    print(average_similarity_score)
    print("\n" + "="*50 + "\n")

    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    del model_test

print("Model fine-tuning complete. Model saved to `output_path_to_save_model`.")

/home/ram/Codes/college/chatbot/local-langchain/venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


fine tune epoch 1


AttributeError: 'OneCycleLR' object has no attribute 'lower'

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import LoggingHandler
import logging
import os
import torch
from models import ModelName, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Setup logging
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO, handlers=[LoggingHandler()])

# File paths and model info
csv_path = 'fine_tuning_dataset/all_merged_dataset.csv'
model_name = ModelName.MULTILINGUAL_MINILM_FINETUNING_EARLY_STOP.value
output_path = models[model_name]['local_dir']

# Load the dataset
df = pd.read_csv(csv_path)

# Assume the CSV has columns: question, context
examples = [InputExample(texts=[row['question'], row['context']]) for idx, row in df.iterrows()]

# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"Using device: {device}")

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model.to(device)  # Move the model to the GPU if available

# Convert the dataset to a DataLoader
train_dataloader = DataLoader(examples, shuffle=True, batch_size=16)

# Define the loss function
train_loss = losses.MultipleNegativesRankingLoss(model)

# Evaluation during training using EmbeddingSimilarityEvaluator
sentences1 = df['question'].tolist()
sentences2 = df['context'].tolist()
evaluator = EmbeddingSimilarityEvaluator(sentences1, sentences2, [1.0]*len(sentences1))  # Assuming 1.0 similarity score for all pairs

# Custom EarlyStopping class
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, score, model, model_path):
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                logging.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, score, model, model_path):
        '''Saves model when evaluation score improves.'''
        if self.verbose:
            logging.info(f'Evaluation score improved to {score}; saving model to {model_path}')
        model.save(model_path)


# Early stopping callback
early_stopping = EarlyStopping(patience=5, verbose=True)

# Make sure output path exists
os.makedirs(output_path, exist_ok=True)

# Train the model with early stopping
epoch = 0
while epoch < 50:
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=1,
              evaluator=evaluator,
              evaluation_steps=1000,
              output_path=output_path,
              show_progress_bar=True)
    
    # Evaluate the model
    score = evaluator(model, output_path)
    print(score)

    early_stopping(score, model, output_path)
    if early_stopping.early_stop:
        logging.info("Early stopping")
        break

    epoch += 1

logging.info("Training finished")

# Save the final model
model.save(os.path.join(output_path, 'fine_tuned_model'))

/home/ram/Codes/college/chatbot/local-langchain/venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


2024-06-21 22:06:01,126 - Using device: cuda
2024-06-21 22:06:01,126 - Use pytorch device_name: cuda
2024-06-21 22:06:01,126 - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


/home/ram/Codes/college/chatbot/local-langchain/venv/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/44 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [ ]:
# # Example evaluation - you may need to adapt according to your dataset
# evaluation_examples = [
#     ("How are you?", "How do you do?"),
#     ("What is your name?", "What's your name?"),
#     ("Where do you live?", "Where is your home located?")
# ]

# model = SentenceTransformer(output_path)

# for pair in evaluation_examples:
#     embeddings = model.encode(pair)
#     similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])
#     print(f"Similarity between: '{pair[0]}' and '{pair[1]}' is {similarity.item():.4f}")